##### MNIST with CNN

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [49]:
from tensorflow.examples.tutorials.mnist import input_data

In [50]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


### Helper Functions

Function to help intialize random weights for fully connected or convolutional layers, we leave the shape attribute as a parameter for this.

In [51]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

Same as init_weights, but for the biases

In [52]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=[1])
    return tf.Variable(init_bias_vals)

Create a 2D convolution using builtin conv2d from TF. From those docs:

Computes a 2-D convolution given 4-D `input` and `filter` tensors.

Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
and a filter / kernel tensor of shape
`[filter_height, filter_width, in_channels, out_channels]`, this op
performs the following:

1. Flattens the filter to a 2-D matrix with shape
   `[filter_height * filter_width * in_channels, output_channels]`.
2. Extracts image patches from the input tensor to form a *virtual*
   tensor of shape `[batch, out_height, out_width,filter_height * filter_width * in_channels]`.
3. For each patch, right-multiplies the filter matrix and the image patch
   vector.


In [53]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

Create a max pooling layer, again using built in TF functions:

Performs the max pooling on the input.

    Args:
      value: A 4-D `Tensor` with shape `[batch, height, width, channels]`and
        type `tf.float32`.
      ksize: A list of ints that has length >= 4.  The size of the window for
        each dimension of the input tensor.
      strides: A list of ints that has length >= 4.  The stride of the sliding
        window for each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. 

In [54]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

Using the conv2d function, we'll return an actual convolutional layer here that uses an ReLu activation.

In [55]:
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

This is a normal fully connected layer

In [56]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    print('from normal full layer',input_size)
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

### Placeholders

In [57]:
x = tf.placeholder(tf.float32,shape=[None,784])

In [58]:
y_true = tf.placeholder(tf.float32,shape=[None,10])

### Layers

In [59]:
x_image = tf.reshape(x,[-1,28,28,1])
x_image

<tf.Tensor 'Reshape_8:0' shape=(?, 28, 28, 1) dtype=float32>

In [60]:
# Using a 6by6 filter here, used 5by5 in video, you can play around with the filter size
# You can change the 32 output, that essentially represents the amount of filters used
# You need to pass in 32 to the next input though, the 1 comes from the original input of 
# a single image.
convo_1 = convolutional_layer(x_image, shape=[6,6,1,32])
convo_1_pooling = max_pool_2by2(convo_1)
convo_1_pooling

<tf.Tensor 'MaxPool_24:0' shape=(?, 14, 14, 32) dtype=float32>

In [73]:
# Using a 6by6 filter here, used 5by5 in video, you can play around with the filter size
# You can actually change the 64 output if you want, you can think of that as a representation
# of the amount of 6by6 filters used.
convo_2 = convolutional_layer(convo_1_pooling,shape=[6,6,32,64])
convo_2_pooling = max_pool_2by2(convo_2)


convo_2_pooling.get_shape()[1]*convo_2_pooling.get_shape()[3]
n = convo_2_pooling.get_shape()
g = n[1:4].num_elements()
g
(n[1])
#convo_2

Dimension(7)

In [62]:
# Why 7 by 7 image? Because we did 2 pooling layers, so (28/2)/2 = 7
# 64 then just comes from the output of the previous Convolution
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,3))
#full_layer_one
convo_2_flat

from normal full layer 3136


<tf.Tensor 'Reshape_9:0' shape=(?, 3136) dtype=float32>

In [74]:
# NOTE THE PLACEHOLDER HERE!
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)
full_one_dropout

<tf.Tensor 'dropout_2/mul:0' shape=(?, 3) dtype=float32>

In [64]:
y_pred = normal_full_layer(full_one_dropout,10)

from normal full layer 3


### Loss Function

In [65]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

### Optimizer

In [66]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

### Intialize Variables

In [67]:
init = tf.global_variables_initializer()

In [68]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    batch_x , batch_y = mnist.train.next_batch(50)
        
    sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
    #p = sess.run(x, feed_dict = {x:batch_x})
    
    #p = sess.run(x_image, feed_dict = {x:batch_x})
    
    p = sess.run(full_layer_one , feed_dict = {x:batch_x})
    
    print((p).shape)
    
    #plt.imshow(p[1])
    #plt.show()
       
    # Test the Train Model
    #matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

    acc = tf.reduce_mean(tf.cast(matches,tf.float32))

    #print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
    #print('\n')

(50, 3)


### Session

steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

## Great Job!